# Proyecto 1: Brecha Digital
### Preparación de datos

#### Basado en el reporte ["La Brecha Digital en California"](https://www.ppic.org/wp-content/uploads/jtf-californias-digital-divide-in-spanish.pdf)

## Pregunta(s) de Investigación:
1. ¿Qué porcentaje de hogares en el estado X tiene acceso a internet de alta velocidad?
2. ¿Varía este número según los grupos demográficos? (en este caso raza/etnia.

## Meta:
* Explorar los archivos de datos (`acsdata.data.gz`) y crear un _conjunto de datos analítico_ (un archivo derivado enfocado específicamente en análisis en mano).

## Contexto:
Obtenimos datos de la encuesta American Community Survey (ACS) de [IPUMS](https://usa.ipums.org/usa/). <br>
Contiene características demográficas:
  - edad
  - sexo
  - raza/etnia

e indicadores geográficos:
  - estado
  - condado

***

#### Paso 1: Prepara tu entorno de trabajo.

**Import**a las bibliotecas necesarias y crea objetos `Path` (ruta de archivo). Esto grarantiza reproducibilidad en distintos sistemas operativos (Windows utiliza `\` en lugar de `/` para separar los nombres de archivos.

Necesitamos: 
1. `pandas` para trabajar con los datos.
2. `pathlib`, y más específicamente su objeto `Path`, para trabajar con rutas de archivos. Esto asegura que nuestro código funcione en Windows (que utiliza `\` en sus rutas) y MacOS/Linux (los cuales utilizan `/`).
3. `datetime` - tip: Existen sistemas de control de versiones para datos pero etiquetar tus archivos de datos (cuando no son masivos) no es un mal primer paso si estás comenzando.

In [1]:
# Prepara tu entorno de trabajo
import pandas as pd
from pathlib import Path
from datetime import datetime as dt
hoy = dt.today().strftime("%d-%m-%y")

print(hoy)

06-05-19


_nota: Aunque estés utilizando Windows puedes escribir_ `/` _en tus rutas de archivo._

In [2]:
# Directorio de datos y rutas
RUTA_DATOS_EN_BRUTO = Path("../datos/brutos/")
RUTA_DATOS_INTERINOS = Path("../datos/interinos/")
RUTA_DATOS_PROCESADOS = Path("../datos/procesados/")
RUTA_DATOS_FINALES = Path("../datos/finales/")

**NOTA:** He incluido un _script_ de `python` llamado `herramientas.py` con la función `arbol` la cual muestra el árbol de directorios. La obtuvé de el [tutorial de RealPython's sobre el módulo `pathlib`](https://realpython.com/python-pathlib/)).

    de nuestro script herramientas importa arbol para utilizarla

In [3]:
from herramientas import arbol

In [4]:
arbol(RUTA_DATOS_EN_BRUTO)

+ ..\datos\brutos
    + acs_data.csv.gz
    + acs_data.dta.gz


***

#### Paso 2: Carga y explora los datos

Con `pandas` _leer_ archivos de datos es tan fácil como escribir `.read_csv(RUTA_AL_ARCHIVO_CSV)` y esto es suficiente la mayoría del tiempo. La función `read_csv()` de `Pandas` es tan poderosa que incluso lee archivos comprimidos sin tener que especificar algún otro parametro. Prueba lo siguiente:

```python
datos = pd.read_csv(RUTA_DATOS_EN_BRUTO / 'acs_data.csv.gz')
datos.head()
```
_Asegurate de cambiar_ `RUTA_DE_DATOS_EN_BRUTO` _a lo que seaque hayas llamado tu variable a la que asignaste el objeto `Path` con la ruta a tus datos en bruto._

In [5]:
datos = pd.read_csv(RUTA_DATOS_EN_BRUTO / 'acs_data.csv.gz')
datos.head()

,YEAR,SERIAL,HHWT,STATEFIP,COUNTYFIP,GQ,CINETHH,CIHISPEED,PERNUM,PERWT,RELATE,RELATED,SEX,AGE,RACE,RACED,HISPAN,HISPAND
0,2017,1,206,1,0,1,3,0,1,206,1,101,1,73,2,200,0,0
1,2017,2,45,1,0,1,1,20,1,45,1,101,2,31,1,100,0,0
2,2017,3,136,1,0,1,1,20,1,136,1,101,1,41,1,100,1,100
3,2017,3,136,1,0,1,1,20,2,121,2,201,2,48,1,100,0,0
4,2017,3,136,1,0,1,1,20,3,111,3,301,1,16,1,100,1,100


***
IPUMS ofrece algunos formatos de datos que pueden ser aún más útiles [[docs]](https://usa.ipums.org/usa-action/faq#ques12):
> Además del archivo de datos ASCII, el sistema crea un archivo de sintaxis de paquetes estadísticos para acompañar cada extracto. El archivo de sintaxis está diseñado para leer los datos ASCII mientras se aplican las variables de valor y las etiquetas apropiadas. SPSS, SAS y Stata son compatibles. Debe descargar el archivo de sintaxis con el extracto o no podrá leer los datos. El archivo de sintaxis requiere una edición menor para identificar la ubicación del archivo de datos en su computadora local.

En este caso, usaremos un archivo **Stata** (`.dta`). La razón principal es que los archivos `.dta` pueden almacenar *etiquetas de valor* que` pandas` luego puede leer y convertir columnas en columnas `Categóricas` en nuestro marco de datos de pandas. Esto 1) ahorra memoria, y 2) es una buena práctica porque ciertas ciencias sociales en serio, en serio _en serio_, y en serio _en serio_ ***en serio*** aman a Stata, por lo que sus conjuntos de datos interesantes son probablemente archivos `.dta`.

Sin embargo, `pandas` no puede leer` .dta` comprimido directamente como lo hace con los archivos `.csv`. Afortunadamente, IPUMS utiliza el formato comprimido *gzip* y `python` incluye un módulo` gzip` en su biblioteca estándar.

**Import**a gzip y prueba lo siguiente:
```python
with gzip.open(RUTA_DATOS_EN_BRUTO / 'acs_data.dta.gz') as archivo:
    datos = pd.read_stata(archivo)
```

y muestra las primeras cinco filas de tu `datos` _DataFrame_.

In [6]:
# importa gzip y carga tus datos
import gzip
with gzip.open(RUTA_DATOS_EN_BRUTO / 'acs_data.dta.gz') as archivo:
    datos = pd.read_stata(archivo)

In [7]:
# muestra las primeras 5 filas
datos.head()

,year,serial,hhwt,statefip,countyfip,gq,cinethh,cihispeed,pernum,perwt,relate,related,sex,age,race,raced,hispan,hispand
0,2017,1,206,alabama,0,households under 1970 definition,"no internet access at this house, apartment, o...",n/a (gq),1,206,head/householder,head/householder,male,73,black/african american/negro,black/african american/negro,not hispanic,not hispanic
1,2017,2,45,alabama,0,households under 1970 definition,"yes, with a subscription to an internet service",no,1,45,head/householder,head/householder,female,31,white,white,not hispanic,not hispanic
2,2017,3,136,alabama,0,households under 1970 definition,"yes, with a subscription to an internet service",no,1,136,head/householder,head/householder,male,41,white,white,mexican,mexican
3,2017,3,136,alabama,0,households under 1970 definition,"yes, with a subscription to an internet service",no,2,121,spouse,spouse,female,48,white,white,not hispanic,not hispanic
4,2017,3,136,alabama,0,households under 1970 definition,"yes, with a subscription to an internet service",no,3,111,child,child,male,16,white,white,mexican,mexican


***

#### Paso 3: Familiarizate con el conjunto de datos.
Ya hemos visto `.head()` - el método de `pandas` que mostrará las primeras 5 filas de tu DataFrame. Esto te da una idea de cómo se ven sus datos. Sin embargo, hay mucho más `.info()` que puede salir de su marco de datos. También puede simplemente pedirle al DataFrame si los `.describe()`...

In [8]:
# averigua más info de tu DataFrame
datos.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3190040 entries, 0 to 3190039
Data columns (total 18 columns):
year         category
serial       int32
hhwt         int16
statefip     category
countyfip    int16
gq           category
cinethh      category
cihispeed    category
pernum       int8
perwt        int16
relate       category
related      category
sex          category
age          category
race         category
raced        category
hispan       category
hispand      category
dtypes: category(13), int16(3), int32(1), int8(1)
memory usage: 100.4 MB


In [9]:
# describe tus datos
datos.describe()

,serial,hhwt,countyfip,pernum,perwt
count,3.190040e+06,3.190040e+06,3.190040e+06,3.190040e+06,3.190040e+06
mean,6.918400e+05,9.614892e+01,5.099185e+01,2.083905e+00,1.021050e+02
std,4.064117e+05,7.594648e+01,8.809394e+01,1.340533e+00,8.314945e+01
min,1.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00
25%,3.350008e+05,5.100000e+01,0.000000e+00,1.000000e+00,5.300000e+01
50%,6.925320e+05,7.600000e+01,1.900000e+01,2.000000e+00,8.000000e+01
75%,1.047493e+06,1.170000e+02,7.300000e+01,3.000000e+00,1.240000e+02
max,1.394399e+06,2.401000e+03,8.100000e+02,2.000000e+01,2.401000e+03


Comprueba la 'forma' de tus datos con su atributo `.shape`. Note la falta de paréntesis.

In [10]:
datos.shape

(3190040, 18)

***

#### Paso 4: Recorta tus datos
En este momento estás trabajando con tu **archivo maestro**: un conjunto de datos que contiene todo lo que _podría_ necesita para su análisis. Realmente no deseas modificar este conjunto de datos porque podría estar usándolo para otros análisis. Por ejemplo, vamos a analizar el acceso a Internet de alta velocidad en un estado de tu elección, pero la próxima semana es posible que desees realizar el mismo análisis en otro estado o tal vez solo en un condado específico. Para asegurarse de que puedas **reutilizar** tus datos y tu código más adelante, creamos un _archivo de análisis_, un conjunto de datos que contiene solo los datos necesarios para **este** análisis específico a mano.

Primero, solo estamos interesados en encontrar la _ "Brecha Digital" de un estado en este momento. El **archivo maestro** contiene datos de los 50 estados y el distrito de Columbia.

Lo que quieres hacer es encontrar todas las filas donde el `statefip` coincide con el nombre de tu estado. Esto se llama indexación booleana.

Prueba lo siguiente:
```python
datos['statefip'] == 'ohio'
```
_Nota: Tu puedes cambiar 'ohio' a cualquiera de los 50 estados o 'district of columbia' para DC._

In [11]:
# Prueba indexación booleanda
datos['statefip'] == 'ohio'

0          False
1          False
2          False
3          False
4          False
5          False
6          False
7          False
8          False
9          False
10         False
11         False
12         False
13         False
14         False
15         False
16         False
17         False
18         False
19         False
20         False
21         False
22         False
23         False
24         False
25         False
26         False
27         False
28         False
29         False
           ...  
3190010    False
3190011    False
3190012    False
3190013    False
3190014    False
3190015    False
3190016    False
3190017    False
3190018    False
3190019    False
3190020    False
3190021    False
3190022    False
3190023    False
3190024    False
3190025    False
3190026    False
3190027    False
3190028    False
3190029    False
3190030    False
3190031    False
3190032    False
3190033    False
3190034    False
3190035    False
3190036    False
3190037    Fal

Esto devolverá un `pandas.Series` de booleanos (Trues y Falses) que luego puede usar para filtrar las filas innecesarias.

Es una buena práctica guardar esta _Serie_ como una variable al principio de tu código (si la conoces de antemano) o justo antes de usarla en caso de que use estas condiciones en más de un lugar. Esto te ahorrará tiempo si decide cambiar el valor que está comparando, `'ohio'` para`' california'` por ejemplo.

```python
mascara_estado = (datos['statefip'] == 'ohio')
datos[mascara_estado].head()
```

In [12]:
# hazlo tu
mascara_estado = (datos['statefip'] == 'ohio')
datos[mascara_estado].head()

,year,serial,hhwt,statefip,countyfip,gq,cinethh,cihispeed,pernum,perwt,relate,related,sex,age,race,raced,hispan,hispand
2185099,2017,953654,55,ohio,0,households under 1970 definition,"no internet access at this house, apartment, o...",n/a (gq),1,55,head/householder,head/householder,male,79,white,white,not hispanic,not hispanic
2185100,2017,953654,55,ohio,0,households under 1970 definition,"no internet access at this house, apartment, o...",n/a (gq),2,69,spouse,spouse,female,79,white,white,not hispanic,not hispanic
2185101,2017,953655,82,ohio,49,households under 1970 definition,"yes, without a subscription to an internet ser...",n/a (gq),1,83,head/householder,head/householder,male,28,three or more major races,"white, black, aian",not hispanic,not hispanic
2185102,2017,953656,160,ohio,0,households under 1970 definition,"yes, with a subscription to an internet service","yes (cable modem, fiber optic or dsl service)",1,159,head/householder,head/householder,female,78,white,white,not hispanic,not hispanic
2185103,2017,953656,160,ohio,0,households under 1970 definition,"yes, with a subscription to an internet service","yes (cable modem, fiber optic or dsl service)",2,169,child,child,female,52,white,white,not hispanic,not hispanic


Guardemoslo en una variable con un nombre más útil:

```python
datos_estatales = datos[mascara_estado].copy()
```

Tienes que utilizar `.copy()` para crear copias _reales_ de los datos. Si ejecutas:
```python
datos_estatales = datos[mascara_estado]
```
`datos_estatales` sería lo que la documentación de `pandas` se refiere como una _vista_ de DataFrame `datos`. Esto puede tener consecuencias más adelante si modificas estos DataFrames. Muchas veces recibirás solo una advertencia y tu código se ejecutará tal como lo esperabas - pero, ¿para qué tomar riesgos?

In [13]:
# asigna tu nuevo DataFrame a la variable datos_estatales
datos_estatales = datos[mascara_estado].copy()

Ahora, veamos que `.columns` tenemos en nuestro DataFrame. Puedes acceder esta información de la misma manera que encontramos la _"forma"_ hace un rato.

In [14]:
datos_estatales.columns

Index(['year', 'serial', 'hhwt', 'statefip', 'countyfip', 'gq', 'cinethh',
       'cihispeed', 'pernum', 'perwt', 'relate', 'related', 'sex', 'age',
       'race', 'raced', 'hispan', 'hispand'],
      dtype='object')

¿Hay columnas de las cuales **tienes seguridad** que no vas a necesitar? <br> Si no estás 90% seguro de que no vas a necesitar una columna, no te deshagas de ella.

Eliminar columnas es tan fácil como utilizar `.drop()` en tu DataFrame.

```python
datos_estatales.drop(columns = ['lista', 'de', 'columnas', 'pa', 'eliminar'])
```

In [15]:
datos_estatales.drop(columns = ['related', 'raced', 'hispand'])

,year,serial,hhwt,statefip,countyfip,gq,cinethh,cihispeed,pernum,perwt,relate,sex,age,race,hispan
2185099,2017,953654,55,ohio,0,households under 1970 definition,"no internet access at this house, apartment, o...",n/a (gq),1,55,head/householder,male,79,white,not hispanic
2185100,2017,953654,55,ohio,0,households under 1970 definition,"no internet access at this house, apartment, o...",n/a (gq),2,69,spouse,female,79,white,not hispanic
2185101,2017,953655,82,ohio,49,households under 1970 definition,"yes, without a subscription to an internet ser...",n/a (gq),1,83,head/householder,male,28,three or more major races,not hispanic
2185102,2017,953656,160,ohio,0,households under 1970 definition,"yes, with a subscription to an internet service","yes (cable modem, fiber optic or dsl service)",1,159,head/householder,female,78,white,not hispanic
2185103,2017,953656,160,ohio,0,households under 1970 definition,"yes, with a subscription to an internet service","yes (cable modem, fiber optic or dsl service)",2,169,child,female,52,white,not hispanic
2185104,2017,953656,160,ohio,0,households under 1970 definition,"yes, with a subscription to an internet service","yes (cable modem, fiber optic or dsl service)",3,190,grandchild,male,26,white,not hispanic
2185105,2017,953656,160,ohio,0,households under 1970 definition,"yes, with a subscription to an internet service","yes (cable modem, fiber optic or dsl service)",4,174,other non-relatives,male,54,white,not hispanic
2185106,2017,953657,24,ohio,45,households under 1970 definition,"yes, with a subscription to an internet service","yes (cable modem, fiber optic or dsl service)",1,24,head/householder,female,54,white,not hispanic
2185107,2017,953657,24,ohio,45,households under 1970 definition,"yes, with a subscription to an internet service","yes (cable modem, fiber optic or dsl service)",2,21,spouse,male,55,white,not hispanic
2185108,2017,953657,24,ohio,45,households under 1970 definition,"yes, with a subscription to an internet service","yes (cable modem, fiber optic or dsl service)",3,31,child,female,28,white,not hispanic


Si hay columnas que tu _crees_ que no vas a necesitar pero no tienes mucha seguridad de que ese es el caso, deberías explorarlas.

Las columnas de los DataFrames de `pandas` son `pandas.Series` y tienen métodos y atributos como los DataFrames.

Exploremos la columna `gq` que representa los valores de la variable `Group Quarters`. La [documentación](https://usa.ipums.org/usa-action/variables/GQ#description_section) de IPUMS define _Group Quarters_ así:

>Group quarters son en su mayoría instituciones y otras viviendas para grupos, por ejemplo, barracas militares y dormitorios.

Exploremos que valores _unicos_ (`.unique()`) tiene la Serie `datos_estatales['gq']`...

In [16]:
datos_estatales['gq'].unique()

[households under 1970 definition, additional households under 1990 definition, additional households under 2000 definition, other group quarters, group quarters--institutions]
Categories (5, object): [households under 1970 definition < additional households under 1990 definition < group quarters--institutions < other group quarters < additional households under 2000 definition]

Tambien podemos ver las cuentas totales de los sus valores (`.value_counts()`) lo que nos daría una mejor idea de que tan útil esta columna podría ser.<br>
Por ejemplo, si una columna tiene 2 valores pero 99% de sus observaciones tiene un valor y el resto tiene otro - en este caso, podrías deshacerte de esa columna ya que no agregaría mucho valor a tu análisis.

<i><span style="font-size:12px"> Hay algunas variables donde el 100% de sus filas tienen el mismo valor \*tos\* \*tos\* </i><span style="font-family:monospace">datos_estatales['year'] </span>

In [17]:
datos_estatales['gq'].value_counts()

households under 1970 definition               112619
other group quarters                             3032
group quarters--institutions                     3022
additional households under 1990 definition       149
additional households under 2000 definition        20
Name: gq, dtype: int64

De la [documentación](https://usa.ipums.org/usa-action/variables/GQ#comparability_section) de IPUMS:
>Hay tres definiciones un poco diferentes de _group quarters_ en IPUMS. Para el periodo 1940-1970 (excluyendo el conjunto de datos 100% de 1940), _group quarters_ eran unidades de vivienda con 5 o más individuos sin relación a la cabeza de la vivienda. Antes de 1940 y en 1980-1990, unidades con 10 o más individuos sin relación a la cabeza de la vivienda eran considerados _group quarters_. **En el censo del 2000, 2010, la ACS y la PRCS, ningún límite fue aplicado; para que un hogar fuera considerado _group quarter_, tenía que estar en una lista de _group quarters_ que el Censo mantiene continuamente. En años pasados, una lista similar fue utilizada, con la regla sobre personas-sin-relación immpuesta como seguridad.** 

Por esta razón, y por el hecho de que la gran mayoría de nuestras observaciones caen bajo la definición de 1970 y 1990, estas son a las que nos apegaremos para nuestro análisis.

Creemos otra _mascara_ para filtrar todos los hogares que no caen bajo nuestra definición

Para evaluar multiples condiciones utilizamos los operadores `&` y `|` ("**Y**" y "__O__", respectivamente).

In [18]:
mascara_hogar = ( datos_estatales['gq'] == 'households under 1970 definition' ) | ( datos_estatales['gq'] == 'additional households under 1990 definition')

<details>
    <summary><strong>Sobre columnas categóricas</strong></summary>
    Otro valor agregado al utilizar variables de tipo <i>categórico</i> es que <strong><i>sí</i></strong> es que los valores son ordenados puedes utilizar los operadores <span style="font-family:monospace"> < </span>y <span> > </span> para evaluar condiciones.
    <pre data-language="python">
    mascara_hogar = (datos_estatales['gq'] <= 'additional households under 1990 definition')
    </pre>
    
</details>


<details>
    <summary><i>Nota sobre </i><span style="font-family:monospace">.copy()</span></summary>
    <i>Porque estamos sobreescribiendo</i> <span style="font-family:monospace">datos_estatales</span> <i>no necesitas utilizar</i> <span style="font-family:monospace">.copy()</span> <i>pero no nos afecta utilizarlo. Además, si eres principiante en</i> <span style="font-family:monospace">pandas</span> <i>es buena practica para cuando en serio necesites utilizar</i> <span style="font-family:monospace">.copy()</span>.
</details>

In [19]:
datos_estatales = datos_estatales[mascara_hogar].copy()

Para este momento ya estas muy cerca de crear un `datos_analisis` DataFrame. Hasta ahora:
1. Mantuviste los datos de un solo estado y te deshiciste del resto.
2. Mantuviste los datos de aquellos _hogares_ en los que tenemos interés y te deshiciste del resto.

Nuestra pregunta de investigación 1 es: _"¿Qué porcentage de hogares en el estado X tiene acceso a internet de alta velocidad?"_

Matemáticamente, 
$$ \frac{hogares\ con\ internet\ de\ alta\ velocidad}{hogares\ en\ el\ estado}$$

Tu conjunto de datos `datos_estatales` ahora contiene todo lo que necesitas para encontrar la respuesta.

***

#### Paso 5: Guarda tus datos
Ahora que haz recordado tu **archivo maestro** en un conjunto de datos enfocado para tu análisis, deberías guardarlo.

Hemos estado trabajando con un archivo `.dta` y sale mejor mantenerlo así.

Prueba lo siguiente:
```python
datos_estatales.to_stata(RUTA_DATOS_INTERINOS / f'datos_estatales-{hoy}.dta', write_index = False)
```

Un par de cosas:
1. Estamos utilizando `f-strings` para etiquetar nuestro archivo de datos con la fecha de hoy.
2. Estamos apagando la bandera `write_index` para que al grabar nuestro archivo no se le agregue una columna "index" (índice). Es nuestro conjunto de datos el índice no tiene valor. En otros análisis puede que tengas un índice que tenga algún valor o significado - en esos casos no vas a querer _apagar_ esta bandera.

In [20]:
datos_estatales.to_stata(RUTA_DATOS_INTERINOS / f'datos_estatales-{hoy}.dta', write_index = False)

***

#### Paso 6: Bonus
¿Qué tal si cambiaramos nuestra pregunta de investigación un poco? de <br> _"¿Qué porcentaje de hogares en el estado X tienen acceso a internet de alta velocidad?_" <br> a <br> _"¿Qué porcentaje de hogares **con niños y niñas de edad escolar** en el estado X tiene acceso a internet de alta velocidad?"_

Esto sería una estadística interecante para los creadores de políticas públicas, especialmente si encontramos diferencias entre grupos demográficos (pregunta de investigación 2).

El reto aquí es que nuestra **unidad de observación** en el archivo `datos_estatales` es una persona (ponderada) y que ahora queremos filtrar aquellos **hogares** sin niños o niñas de edad escolar. Esto puede sonar un poco complicado a primera vista pero solo requiere modificar nuestro flujo de trabajo un poco.

Necesitamos hacer unas cuantas cosas:
1. Definir a que nos referimos con _niños/niñas de edad escolar_.
2. Crear una _mascara_ para identificar todos los hogares donde esten estos niños o niñas.
3. Crear una lista sin duplicados de estas identificaciones de hogar (`'serial'`) 
4. Usar esta lista para eliminar todas las observaciones innecesarias.

#### Paso 6.1: Niños/Niñas de edad escolar
La mayoría de las personas concordamos que edad escolar (de primaria a preparatoria) es de 6 a 17 años. Algunas personas están mas interesadas en Kindergarden - Preparatoria (de 5 a 17 o 18 años). Algunas más no incluirían personas de 18 años. Lo importante es saber debes poder defender el porque de que cualquier rango.

Para este análisis, yo sugeriría que usemos 5 - 18 años (Kinder a Preparatoria) pero tu puedes escoger cualquier rango que desees. ¿Qué tal edad de preparatoria (14-18)? Eso sería interesante, lo más probable es que necesites acceso a internet de alta velocidad en la preparatoria más que cuando estas en kindergarden.

In [21]:
mascara_ninxs = (datos_estatales['age'] >= '5') & (datos_estatales['age'] <= '18')

<details>
    <summary><i>¿Qué tipo de datos es la Serie</i> <span style='font-family:monospace'>datos_estatales['age']</span> <i>?</i> </summary>
    Categórica. Esto significa que aunque los valores de esta columna se <i>vean</i> como números, la realidad es que son <i>etiquetas de valor</i> las cuales son <i>cadenas de caracteres</i> o <i>"strings"</i>.
</details>

Ahora que tenemos nuestra _máscara_ podemos utilizarla para crear una lista de hogares con niños/niñas.

Anteriormente, aplicamos una _máscara_ a un DataFrame y la guardamos en una variable. Ahora, vamos a dar un paso más y tomar una sola columna del marco de datos _filtrado_.

Intentalo tu primero.

<details>
    <summary>sugerencia</summary>
        <i>¿Cómo tomamos y exploramos una sola columna de un DataFrame anteriormente?</i>
</details>

In [22]:
hogares_con_ninxs = datos_estatales[mascara_ninxs]['serial'].copy()

In [23]:
hogares_con_ninxs.head()

2185119    953662
2185129    953668
2185137    953671
2185138    953671
2185165    953685
Name: serial, dtype: int32

¿Recuerdas como eliminamos (`drop`) columnas al inicio?

¿Cómo crees que podemos eliminar duplicados?

<details>
    <summary>respuesta</summary>
    <pre>.drop_duplicates()</pre>
</details>

In [24]:
hogares_con_ninxs = hogares_con_ninxs.drop_duplicates()

Ahora que tienes una lista de valores únicos de los hogares con niños/niñas, solo tienes que revisar si un valor de la columna `serial` de nuestro marco de datos `datos_estatales` también en nuestra series `hogares_con_ninxs`.


<details>
    <summary>respuesta</summary>
    <details>
        <summary>minimo intentalo</summary>
        <details>
            <summary>bueno pues</summary>
            <details>
                <summary>en inglés como dirías <i>"está en"</i></summary>
                <span style="font-family:monospace">.isin()</span>, obvis.
            </details>
        </details>
    </details>
</details>

In [25]:
mascara_hogares_con_ninxs = datos_estatales['serial'].isin(hogares_con_ninxs)
datos_estatales[mascara_hogares_con_ninxs]

,year,serial,hhwt,statefip,countyfip,gq,cinethh,cihispeed,pernum,perwt,relate,related,sex,age,race,raced,hispan,hispand
2185117,2017,953662,57,ohio,0,households under 1970 definition,"yes, with a subscription to an internet service","yes (cable modem, fiber optic or dsl service)",1,58,head/householder,head/householder,female,48,white,white,not hispanic,not hispanic
2185118,2017,953662,57,ohio,0,households under 1970 definition,"yes, with a subscription to an internet service","yes (cable modem, fiber optic or dsl service)",2,62,child,child,male,20,white,white,not hispanic,not hispanic
2185119,2017,953662,57,ohio,0,households under 1970 definition,"yes, with a subscription to an internet service","yes (cable modem, fiber optic or dsl service)",3,78,child,child,female,9,white,white,not hispanic,not hispanic
2185128,2017,953668,140,ohio,61,households under 1970 definition,"yes, with a subscription to an internet service","yes (cable modem, fiber optic or dsl service)",1,140,head/householder,head/householder,male,28,black/african american/negro,black/african american/negro,not hispanic,not hispanic
2185129,2017,953668,140,ohio,61,households under 1970 definition,"yes, with a subscription to an internet service","yes (cable modem, fiber optic or dsl service)",2,192,sibling,sibling,female,16,black/african american/negro,black/african american/negro,not hispanic,not hispanic
2185130,2017,953668,140,ohio,61,households under 1970 definition,"yes, with a subscription to an internet service","yes (cable modem, fiber optic or dsl service)",3,161,parent,parent,female,48,black/african american/negro,black/african american/negro,not hispanic,not hispanic
2185136,2017,953671,135,ohio,0,households under 1970 definition,"yes, with a subscription to an internet service",no,1,134,head/householder,head/householder,female,35,black/african american/negro,black/african american/negro,not hispanic,not hispanic
2185137,2017,953671,135,ohio,0,households under 1970 definition,"yes, with a subscription to an internet service",no,2,115,child,child,male,7,black/african american/negro,black/african american/negro,not hispanic,not hispanic
2185138,2017,953671,135,ohio,0,households under 1970 definition,"yes, with a subscription to an internet service",no,3,117,child,child,male,7,black/african american/negro,black/african american/negro,not hispanic,not hispanic
2185163,2017,953685,46,ohio,35,households under 1970 definition,"yes, with a subscription to an internet service","yes (cable modem, fiber optic or dsl service)",1,45,head/householder,head/householder,male,56,white,white,not hispanic,not hispanic


Grabemos esto como `datos_para_analisis` en nuestra computadora.

In [26]:
datos_para_analisis = datos_estatales[mascara_hogares_con_ninxs]

```python
datos_para_analisis.to_stata(RUTA_DATOS_INTERINOS / f'datos_para_analisis-{hoy}.dta', write_index = False)
```

In [27]:
datos_para_analisis.to_stata(RUTA_DATOS_INTERINOS / f'datos_para_analisis-{hoy}.dta', write_index = False)